In [1]:
#%%
# system level
import os
from os import path
import sys
import argparse


# deep learning
from scipy.stats import pearsonr, spearmanr
import numpy as np
import torch
from torch import nn
from torchvision import models,transforms
import torch.optim as optim
import wandb
from sklearn.model_selection import GroupKFold
from sklearn.linear_model import LinearRegression

# data 
import pandas as pd
import cv2
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
import plotly.express as px
import plotly.graph_objects as go
from scipy.optimize import curve_fit

# local
from nerf_qa.DISTS_pytorch.DISTS_pt import DISTS, prepare_image
from nerf_qa.data import LargeQADataset
from nerf_qa.settings import DEVICE_BATCH_SIZE

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')




DATA_DIR = "/home/ccl/Datasets/NeRF-QA"
REF_DIR = path.join(DATA_DIR, "Reference")
SYN_DIR = path.join(DATA_DIR, "NeRF-QA_videos")
SCORE_FILE = path.join(DATA_DIR, "NeRF_VQA_MOS.csv")

# Read the CSV file
scores_df = pd.read_csv(SCORE_FILE)
train_df = pd.read_csv("/home/ccl/Datasets/NeRF-QA-Large-1/scores.csv")

/tmp/ipykernel_2963825/3491669548.py:21: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
def plot_dists_mos(df):
    x_data = df['MOS']
    y_data = df['DISTS']
    group_data = df['reference_filename']
    def sqrt_func(x, a, b):
        return a + b * x #np.sqrt(x)
    # Fit the model
    params, params_covariance = curve_fit(sqrt_func, x_data, y_data)

    # Predict using the fitted model
    x_range = np.linspace(min(x_data), max(x_data), 400)
    y_pred = sqrt_func(x_range, *params)

    # Plotting
    fig = go.Figure()

    # Color points by group. We need to create a trace for each unique group.
    unique_groups = df['reference_filename'].unique()
    for group in unique_groups:
        group_x = df[df['reference_filename'] == group]['MOS']
        group_y = df[df['reference_filename'] == group]['DISTS']
        fig.add_trace(go.Scatter(x=group_x, y=group_y, mode='markers', name=group))

    # Regression line
    fig.add_trace(go.Scatter(x=x_range, y=y_pred, mode='lines', name='Linear Regression'))

    fig.update_layout(title='Linear Regression between DISTS and MOS',
                      xaxis_title='MOS',
                      yaxis_title='DISTS')
    return fig
fig = plot_dists_mos(scores_df)
display(fig)
display(scores_df.head(3))

,distorted_filename,reference_filename,FVVHD,DISTS,SSIM,GMSD,MOS,DoMOS,DMOS,Buike MOS,...,WaDiQaM_score,CompressVQA,PSNR_Score,MS-SSIM_Score,LPIPS_Score,WaDiQa_score,LPIPS_Score_vgg,PLCC,SROCC,KROCC
0,lego_tensorf.mp4,lego_reference.mp4,9.0069,0.035496,0.99129,0.035686,4.6396,5.000000,-12.1555,5.0,...,0.798469,3.745422,36.646977,0.968563,0.029931,79.846926,0.089946,0.363577,0.474598,0.324468
1,lego_dvgo.mp4,lego_reference.mp4,8.9140,0.025291,0.99086,0.038980,4.5862,4.922538,-10.0190,4.0,...,0.796464,3.848095,36.857570,0.968916,0.026688,79.646396,0.055983,-0.377061,-0.569366,-0.423759
2,lego_tensorf_ss2.mp4,lego_reference.mp4,8.9029,0.037678,0.99005,0.038398,4.3874,4.634879,-2.0850,4.0,...,0.797675,3.738048,36.581300,0.966980,0.031191,79.767545,0.093800,-0.602582,-0.645571,-0.464539


In [3]:


def plot_dists_mos_with_group_regression_b_ave(df, y_col='DISTS', group_col='reference_filename'):
    # Define a list of colors for the groups
    colors = [
        '#1f77b4',  # Mutated blue
        '#ff7f0e',  # Safety orange
        '#2ca02c',  # Cooked asparagus green
        '#d62728',  # Brick red
        '#9467bd',  # Muted purple
        '#8c564b',  # Chestnut brown
        '#e377c2',  # Raspberry yogurt pink
        '#7f7f7f',  # Middle gray
        '#bcbd22',  # Curry yellow-green
        '#17becf'   # Blue-teal
    ]

    def linear_func(x, a, b):
        return a + b * x

    # Plotting
    fig = go.Figure()

    unique_groups = df[group_col].unique()
    for i, group in enumerate(unique_groups):
        group_df = df[df[group_col] == group]
        group_x = group_df['MOS']
        group_y = group_df[y_col]
        
        # Fit the model for each group
        params, params_covariance = curve_fit(linear_func, group_x, group_y)
        
        # Predict using the fitted model for the group
        x_range = np.linspace(min(group_x), max(group_x), 400)
        y_pred = linear_func(x_range, *params)
        
        # Ensure we use a unique color for each group, cycling through the colors list if necessary
        color = colors[i % len(colors)]
        
        # Data points for the group
        fig.add_trace(go.Scatter(x=group_x, y=group_y, mode='markers', name=f'Data: {group}', marker_color=color))
        
        # Regression line for the group
        fig.add_trace(go.Scatter(x=x_range, y=y_pred, mode='lines', name=f'Regression: {group}', line=dict(color=color)))

    fig.update_layout(title=f'Linear Regression per Group between {y_col} and MOS',
                      xaxis_title='MOS',
                      yaxis_title=y_col)
    return fig

display(plot_dists_mos_with_group_regression_b_ave(scores_df, 'DISTS'))
display(plot_dists_mos_with_group_regression_b_ave(train_df, 'DISTS', 'referenced_filename'))

In [6]:


def plot_dists_mos_with_group_regression(df, y_col='DISTS', group_col='reference_filename'):
    # Define a list of colors for the groups
    colors = [
        '#1f77b4',  # Mutated blue
        '#ff7f0e',  # Safety orange
        '#2ca02c',  # Cooked asparagus green
        '#d62728',  # Brick red
        '#9467bd',  # Muted purple
        '#8c564b',  # Chestnut brown
        '#e377c2',  # Raspberry yogurt pink
        '#7f7f7f',  # Middle gray
        '#bcbd22',  # Curry yellow-green
        '#17becf'   # Blue-teal
    ]

    def linear_func(x, a, b):
        return a + b * x

    # Plotting
    fig = go.Figure()

    unique_groups = df[group_col].unique()
    b = []
    for i, group in enumerate(unique_groups):
        group_df = df[df[group_col] == group]
        group_x = group_df['MOS']
        group_y = group_df[y_col]
        # Fit the model for each group
        params, params_covariance = curve_fit(linear_func, group_x, group_y)
        b.append(params[1])

    b_ave = np.mean(b)
    biases = []
        
    for i, group in enumerate(unique_groups):
        group_df = df[df[group_col] == group]
        group_x = group_df['MOS']
        group_y = group_df[y_col]
        
        # Predict using the fitted model for the group
        x_range = np.linspace(min(group_x), max(group_x), 400)
        a = np.mean(group_y.values - group_x.values * b_ave)
        y_pred = linear_func(x_range, a, b_ave)

        biases.append(a)

        
        # Ensure we use a unique color for each group, cycling through the colors list if necessary
        color = colors[i % len(colors)]
        
        # Data points for the group
        fig.add_trace(go.Scatter(x=group_x, y=group_y, mode='markers', name=f'Data: {group}', marker_color=color))
        
        # Regression line for the group
        fig.add_trace(go.Scatter(x=x_range, y=y_pred, mode='lines', name=f'Regression: {group}', line=dict(color=color)))

    fig.update_layout(title=f'Linear Regression per Group between {y_col} and MOS',
                      xaxis_title='MOS',
                      yaxis_title=y_col)
    
    df = pd.DataFrame({
        group_col: unique_groups,
        'bias': biases
    })
    return fig, df

display(plot_dists_mos_with_group_regression(scores_df, 'DISTS')[0])
display(plot_dists_mos_with_group_regression(train_df, 'DISTS', 'referenced_filename')[0])

In [8]:
from nerf_qa.ICNet.ICNet import ICNet, infer_one_image
ICNET_CKPT_PATH = "nerf_qa/ICNet/ck.pth"

icnet_model = ICNet()
icnet_model.load_state_dict(torch.load(ICNET_CKPT_PATH, map_location=torch.device('cpu')))
icnet_model.eval()
icnet_model.to(device)


/home/ccl/.conda/envs/nerf-qa/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning:

The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.

/home/ccl/.conda/envs/nerf-qa/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning:

Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.



ICNet(
  (b1_1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    

In [9]:
test_scenes = ['ship', 'lego', 'drums', 'ficus', 'train', 'm60', 'playground', 'truck']
filtered_train_df = train_df[~train_df['scene'].isin(test_scenes)].reset_index()

_, bias_df = plot_dists_mos_with_group_regression(filtered_train_df, 'DISTS', 'referenced_filename')
display(bias_df)

,referenced_filename,bias
0,gt_hotdog,0.186243
1,gt_materials,0.166145
2,gt_horns,0.193232
3,gt_trex,0.182655


In [14]:

from PIL import Image
import pyfeats

def compute_correlations(pred_scores, mos):
    plcc = pearsonr(pred_scores, mos)[0]
    srcc = spearmanr(pred_scores, mos)[0]
    #ktcc = kendalltau(pred_scores, mos)[0]

    return {
        'plcc': plcc,
        'srcc': srcc,
        #'ktcc': ktcc,
    }


_, bias_df = plot_dists_mos_with_group_regression(train_df, 'DISTS', 'referenced_filename')
bias_df = bias_df.sort_values(by=['bias']).reset_index(drop=True)
for index, row in bias_df.iterrows():
    referenced_filename = row['referenced_filename']
    referenced_path = os.path.join("/home/ccl/Datasets/NeRF-QA-Large-1/references/", referenced_filename, f"{0:03d}.png")
    print(referenced_path)
    referenced_image = np.array(prepare_image(Image.open(referenced_path).convert("L")).squeeze(0).squeeze(0))
    icnet_score = infer_one_image(referenced_path, icnet_model)
    bias_df.at[index, 'icnet_score'] = icnet_score
    #referenced_image = np.array(Image.open(referenced_path).convert("L"))
    #display(referenced_image)
    #features, labels = pyfeats.fdta(referenced_image, np.ones_like(referenced_image), s=3)
    #features, labels = pyfeats.fos(referenced_image, np.ones_like(referenced_image))
    
    #for label, feature in zip(labels, features):
    #    bias_df.at[index, label] = feature


print('icnet_score', compute_correlations(bias_df['icnet_score'], bias_df['bias']))
#for label, feature in zip(labels, features):
#    print(label, compute_correlations(bias_df[label], bias_df['bias']))

display(bias_df)

/home/ccl/Datasets/NeRF-QA-Large-1/references/gt_materials/000.png
/home/ccl/Datasets/NeRF-QA-Large-1/references/gt_trex/000.png
/home/ccl/Datasets/NeRF-QA-Large-1/references/gt_hotdog/000.png
/home/ccl/Datasets/NeRF-QA-Large-1/references/gt_ship/000.png
/home/ccl/Datasets/NeRF-QA-Large-1/references/gt_horns/000.png
/home/ccl/Datasets/NeRF-QA-Large-1/references/gt_drums/000.png
/home/ccl/Datasets/NeRF-QA-Large-1/references/gt_m60/000.png
/home/ccl/Datasets/NeRF-QA-Large-1/references/gt_train/000.png
icnet_score {'plcc': 0.4672378334718216, 'srcc': 0.28571428571428575}


,referenced_filename,bias,icnet_score
0,gt_materials,0.166628,0.360448
1,gt_trex,0.183271,0.590885
2,gt_hotdog,0.186981,0.338810
3,gt_ship,0.188524,0.426152
4,gt_horns,0.193983,0.524879
5,gt_drums,0.211667,0.411899
6,gt_m60,0.293781,0.544766
7,gt_train,0.301722,0.522498


In [17]:
merge_train_df = pd.merge(train_df, bias_df)
filtered_train_df = merge_train_df[~merge_train_df['scene'].isin(test_scenes)].reset_index()
filtered_val_df = merge_train_df[merge_train_df['scene'].isin(test_scenes)].reset_index()
display(filtered_train_df)

,index,scene,frame_count,distorted_filename,referenced_filename,PSNR,MOS,BRISQUE_Score,GMSD,SSIM,DISTS,WaDiQaM-FR_score,LPIPS_Score,Contrique_score,bias,icnet_score
0,0,hotdog,200,tensorf_hotdog_VM,gt_hotdog,37.52,4.420,83.157063,0.025009,0.99329,0.025176,0.799518,0.030874,0.043444,0.186981,0.338810
1,1,hotdog,200,dvgo_hotdog_C-F_51k_102k,gt_hotdog,33.69,2.580,89.197626,0.049309,0.98470,0.061995,0.727710,0.070504,0.163398,0.186981,0.338810
2,2,hotdog,200,instantNGP_hotdog_HM_16,gt_hotdog,37.66,4.180,84.421937,0.025067,0.99437,0.023096,0.789809,0.038980,0.091179,0.186981,0.338810
3,3,hotdog,200,plenoxel_hotdog_SH_dim_1,gt_hotdog,33.38,3.200,81.976803,0.042403,0.98521,0.053716,0.768902,0.049680,0.079081,0.186981,0.338810
4,8,materials,200,tensorf_materials_VM,gt_materials,30.11,2.810,73.393842,0.060510,0.98081,0.043474,0.787615,0.055890,0.067050,0.166628,0.360448
5,9,materials,200,dvgo_materials_C-F_51k_102k,gt_materials,26.00,1.120,70.482765,0.109530,0.94233,0.122838,0.710330,0.119369,0.070791,0.166628,0.360448
6,10,materials,200,instantNGP_materials_HM_16,gt_materials,29.63,3.130,79.407956,0.065370,0.98186,0.052624,0.763429,0.061460,0.070482,0.166628,0.360448
7,11,materials,200,plenoxel_materials_SH_dim_1,gt_materials,25.30,2.040,70.384248,0.104090,0.94232,0.111405,0.727871,0.092428,0.119982,0.166628,0.360448
8,20,materials,200,tensorf_materials_CP_SH_32,gt_materials,27.14,2.000,67.634241,0.095020,0.95549,0.107858,0.736876,0.116354,0.113798,0.166628,0.360448
9,21,materials,200,dvgo_materials_F_2048k,gt_materials,29.38,2.080,75.639004,0.066805,0.97581,0.052796,0.776140,0.060414,0.055748,0.166628,0.360448


In [25]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error


# Step 1: Prepare the Data
X_train = merge_train_df[['DISTS', 'bias']]  # Input features
y_train = merge_train_df['MOS']  # Target variable
X_val = filtered_val_df[['DISTS', 'bias']]  # Input features
y_val = filtered_val_df['MOS']  # Target variable

# Step 3: Create and Train the Model
model = LinearRegression()
model.fit(X_train, y_train)

# Optional: Evaluate the model
y_pred = model.predict(X_val)
mse = mean_squared_error(y_val, y_pred)
print(f'Mean Squared Error: {mse}')
print('icnet_score', compute_correlations(y_pred, y_val))

Mean Squared Error: 0.16395768187937151
icnet_score {'plcc': 0.8849209280052316, 'srcc': 0.8681163859307214}


In [26]:
_, bias_df = plot_dists_mos_with_group_regression(scores_df, 'DISTS', 'reference_filename')
bias_df = bias_df.sort_values(by=['bias']).reset_index(drop=True)
filtered_test_df = pd.merge(scores_df, bias_df, on='reference_filename')
# Step 1: Prepare the Data
X_test = filtered_test_df[['DISTS', 'bias']]  # Input features
y_test = filtered_test_df['MOS']  # Target variable

# Optional: Evaluate the model
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')
print('icnet_score', compute_correlations(y_pred, y_test))

Mean Squared Error: 0.2425895927859271
icnet_score {'plcc': 0.9003702370413762, 'srcc': 0.8637646547980894}
